In [ ]:
import os
import argparse
import numpy as np
from glob import glob
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["SM_FRAMEWORK"] = "tf.keras"
import tensorflow as tf
import models
import metrics
import losses
import dataloaders as dl
from utils import ideatlas, deeplearning
from tqdm.keras import TqdmCallback
import math

In [ ]:
# List available GPUs
physical_devices = tf.config.experimental.list_physical_devices('GPU')

if physical_devices:
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(f"Error setting memory growth: {e}")
else:
    print("No GPUs detected.")

In [ ]:
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
import json
def load_config(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    return config

config = load_config('./config.json')

batch = config["batch"]
epochs = config["epochs"]
data_dir = config['data_dir']
chk_dir = config['chkdir']
log_dir = config["logdir"]
city = data_dir.split('/')[2]
print(city)
inputs = config["dataset"]
inputs_str = "_".join(inputs)

In [ ]:
input_shapes = {input: config["in_shape"][input] for input in inputs}
for input, shape in input_shapes.items():
    # print(f"Shape of {input} is {shape}")
    dim = config["in_shape"][input]
    X,Y = dim[:2]

In [ ]:
print('Training model on', city.capitalize(), 'dataset')
print('Input: ', inputs)
# train_s1 = ideatlas.load_image(data_dir + 'train', X, Y)
train_s2 = ideatlas.load_image(data_dir + 'train', X, Y)
train_bd = ideatlas.load_bd(data_dir + 'train', X, Y)
train_label = ideatlas.load_mask(data_dir + 'train', X, Y, config['n_classes'])

#Validation
# valid_s1 = ideatlas.load_image(data_dir + 'val', X, Y)
valid_s2 = ideatlas.load_image(data_dir + 'val/', X, Y)
valid_bd = ideatlas.load_bd(data_dir + 'val', X, Y)
valid_label = ideatlas.load_mask(data_dir + 'val/', X, Y, config['n_classes'])

In [ ]:
print(f'Training data {train_label.shape, train_s2.shape, train_bd.shape}') 
print(f'Validation data {valid_label.shape, valid_s2.shape, valid_bd.shape}')

In [ ]:
train_s2 = ideatlas.norm_s2(train_s2)
valid_s2 = ideatlas.norm_s2(valid_s2)

# train_images  = [train_s2, train_bd]
# valid_images  = [valid_s2, valid_bd]

train_images = np.concatenate((train_s2, train_bd), axis=-1)
valid_images = np.concatenate((valid_s2, valid_bd), axis=-1)


In [ ]:
print("Min and Max value in train image: ", train_s2.min(), train_s2.max())
print("Labels in the mask are : ", np.unique(train_label))

In [ ]:

# trainData = glob(os.path.join(config['vhr'] + 'train_im', 'HR*.tif'))
# validData = glob(os.path.join(config['vhr'] + 'valid_im', 'HR*.tif'))

# train_datagen = ideatlas.image_generator(trainData, batch_size, (X,Y,4), config['n_classes'], config['vhr'] + 'train_gt')
# validation_datagen = ideatlas.image_generator(validData, batch_size, (X,Y,4), config['n_classes'], config['vhr'] + 'valid_gt')

# ideatlas.patch_class_proportion(train_masks)

In [ ]:
## Randomly visualizing the training images and its corresponding mask
# ideatlas.plot_samples_datagen(train_datagen)
ideatlas.plot_samples_1hot(train_s2, train_label)

In [ ]:
class_weights = ideatlas.calculate_class_weights(train_label)
print(f'Class weight: {class_weights}')

In [ ]:
import segmentation_models as sm

import models.fcn
dice_loss = sm.losses.DiceLoss(class_weights=class_weights) 
focal_loss = sm.losses.CategoricalFocalLoss()
t_loss =  dice_loss + (2 * focal_loss)
j_loss = sm.losses.JaccardLoss(class_weights=class_weights, class_indexes=None, per_image=False, smooth=1e-05)
# model = models.mbcnn(CL=config["n_classes"], input_shapes=input_shapes, dropout_rate=0.2, batch_norm=True, drop_train=False)
# model = segmentation.lightunet(input_shape = (X,Y,4), NUM_CLASSES=3, dropout_rate=0.2, batch_norm=True)
# model = segmentation.mbcnn(CL=config["n_classes"], input_shapes=input_shapes, dropout_rate=0.2, batch_norm=True, drop_train=False)
model = models.fcndk6(input_shapes=(X,Y,11), CL=config["n_classes"])

In [ ]:

model.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate=config["lr"]),
loss=t_loss,
# loss = losses.FocalLoss(),
metrics=[sm.metrics.FScore(threshold=0.5, class_indexes=None, per_image=False, smooth=1e-05, name='f1')]
)
print(f'| Model -> {model.name} | Parameters -> {model.count_params()} |')


In [ ]:
# model.compile(
#     optimizer='adam',
#     loss={
#         'regression': 'mean_squared_error',
#         'segmentation': losses.FocalLoss()
#     },
#     loss_weights={
#         'regression': 1.0,
#         'segmentation': 1.0
#     },
#     metrics={
#         'regression': ['mean_squared_error'],
#         'segmentation': [metrics.IoU()]
#     }
# )

In [ ]:
for i in os.listdir(chk_dir):
    if i.endswith('.h5'):
        os.remove(os.path.join(chk_dir, i))

# Delete .csv files in ./log/
for j in os.listdir(log_dir):
    if j.endswith('.csv'):
        os.remove(os.path.join(log_dir, j))

In [ ]:
# define callbacks for saving logs, model weights, early stopping and learning rate schedule
# steps_per_epoch=len(trainData) // batch_size
steps_per_epoch = len(train_images[0])
callbacks = [
    # deeplearning.LRWarmup(
    #     warmup_steps=steps_per_epoch,
    #     target=config['lr'],
    #     verbose=0,
    # ),
    # tf.keras.callbacks.ReduceLROnPlateau(
    #     monitor="val_loss",
    #     mode="min",
    #     factor=0.1,
    #     patience=10,
    #     verbose=0,
    # ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        mode="min",
        patience=15,
        verbose=1,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(f'./{chk_dir}/{city}_{inputs_str}_{model.name}.weights.h5'),
        monitor=f"val_loss",
        mode="min",
        save_best_only=True,
        save_weights_only=True,
    ),
    tf.keras.callbacks.CSVLogger(
        os.path.join("log", f"{city}_{inputs_str}_{model.name}_log.csv"),
    ),
    deeplearning.training_progress(epochs, steps_per_epoch)
]

In [ ]:
model.fit(train_images, train_label,
        batch_size=batch,
        steps_per_epoch = math.ceil(len(train_s2) / batch),
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(valid_images, valid_label),
        validation_steps = len(valid_s2) // batch,
        verbose=0)

In [ ]:
## 
# Fit with multi task learning
# hist = model.fit(train_images, [train_bd, train_masks],
#                 batch_size = batch_size,
#                 steps_per_epoch = len(train_s2) // batch_size,
#                 epochs=epochs,
#                 callbacks=callbacks, 
#                 validation_data=(valid_images, [valid_bd, valid_masks]),
#                 validation_steps = len(valid_s2) // batch_size)

#Fit with dataloader
# hist = model.fit(train_datagen,
#                     steps_per_epoch=len(trainData) // batch_size,
#                     epochs=epochs,
#                     callbacks=[callbacks],
#                     validation_data=validation_datagen,
#                     validation_steps=len(validData) // batch_size,
#                     verbose=1)

In [ ]:
# # ## Plot training log
import pandas as pd
history = pd.read_csv(os.path.join("log", f"{city}_{inputs_str}_{model.name}_log.csv"))
ideatlas.plot_log(history)

In [ ]:
print('Testing a model')
model = models.mbcnn(CL=config["n_classes"], input_shapes=input_shapes, dropout_rate=0.25, batch_norm=True, drop_train=False)
# model = segmentation.lightunet(input_shape = (X,Y,11), NUM_CLASSES=3, dropout_rate=0.2, batch_norm=True)
# weight =(f'./{chk_dir}/{city}_{inputs_str}_{model.name}_weight.h5')
weight = '/data/experiments-tf/checkpoint/mbcnn_weights/buenos_aires_s2_morph_mbcnn_weight_f58.h5'
print(weight)
model.load_weights(weight)

In [ ]:
#Test
# test_s1 = ideatlas.load_image(config['data_dir'] + 'image/test_s1', X, Y)
test_s2 = ideatlas.load_image(config['data_dir'] + 'test', X, Y)
test_bd = ideatlas.load_bd(config['data_dir'] + 'test', X, Y)
test_label = ideatlas.load_mask(config['data_dir'] + 'test', X, Y, config['n_classes'])

In [ ]:
test_s2 = ideatlas.norm_s2(test_s2)
# test_images = [test_s2, test_bd]
test_images = np.concatenate((test_s2, test_bd), axis=-1)

In [ ]:
print(f'Test data {test_label.shape,  test_s2.shape,  test_bd.shape}')  

In [ ]:
class_report, cm = metrics.class_report(test_images, test_label, model)
# class_report, cm = metrics.class_report_mtcnn(test_images, test_masks, model)
# class_report, cm = metrics.class_report_large_patches(test_images, test_label, model, 128, 128)
print("Classification Report:")
print(class_report)

In [ ]:
# # Plot the confusion matrix
ideatlas.plot_confusion_matrix(cm, save_plot=None)

In [ ]:
# plot_prediction_from_dataloader(test_images, test_label, model)
# ideatlas.plot_prediction(test_images, test_label, model)
# ideatlas.plot_prediction_mbcnn(test_images, test_label, model)

In [ ]:
import matplotlib.colors as mcolors
colors = ['#636363', '#ced2d2', '#fd0006'] #non-builtup, formal, informal
customCmap = mcolors.ListedColormap(colors)
def plot_prediction(test_images, test_label, model):
    test_img_number = random.randint(0, len(test_images)-1)
    print(f'Test image number: {test_img_number}')
    test_img = test_images[test_img_number]
    ground_truth=test_label[test_img_number]
    test_img_input=np.expand_dims(test_img, 0)
    test_pred = model.predict(test_img_input) 
    test_prediction = np.argmax(test_pred, axis=3)[0,:,:] 

    plt.figure(figsize=(20, 12))
    plt.subplot(231)
    plt.title('Test image', fontsize = 25)
    plt.imshow(test_img[..., [2,1,0]]*3)
    plt.subplot(232)
    plt.title('Reference', fontsize = 25)
    plt.imshow(ground_truth[:,:,:], cmap=customCmap, alpha=0.75, vmin=0, vmax=2)
    plt.subplot(233)
    plt.title('Prediction', fontsize = 25)
    plt.imshow(test_prediction, cmap=customCmap, alpha=0.75, vmin=0, vmax=2)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
plot_prediction(test_images, test_label, model)

In [ ]:
# predicted_density, predicted_masks = model.predict(test_images)

In [ ]:
# def mtcnn_inference(N_CLASSES, raster_paths, ndbi_path, model, prediction_path, vector_path):
#     src = rio.open(raster_paths)
#     ndbi_src = rio.open(ndbi_path)

#     ds = src.read()
#     ndbi_ds = ndbi_src.read(1)  # Assuming NDBI is a single band raster

#     large_image = np.moveaxis(ds, 0, -1)
#     large_ndbi = ndbi_ds

#     patch_height, patch_width = model.inputs[0].shape[1], model.inputs[0].shape[2]
#     stride = int(patch_height / 4)  # Set the desired overlap between patches
#     image_height, image_width = large_image.shape[:2]
#     y_pred = np.zeros((image_height, image_width, N_CLASSES))
#     count_map = np.zeros((image_height, image_width, N_CLASSES))

#     total_iterations = ((image_height - patch_height + 1) // stride) * ((image_width - patch_width + 1) // stride)
#     pbar = tqdm(total=total_iterations, desc="Running Full Inference:")

#     for y in range(0, image_height - patch_height + 1, stride):
#         for x in range(0, image_width - patch_width + 1, stride):
#             patch = large_image[y:y + patch_height, x:x + patch_width]
#             ndbi_patch = large_ndbi[y:y + patch_height, x:x + patch_width]

#             # Expand dimensions for model input
#             input_patch = np.expand_dims(patch, axis=0)
#             input_ndbi_patch = np.expand_dims(ndbi_patch, axis=(0, -1))

#             # Predict using the model
#             patch_predictions = model.predict([input_patch, input_ndbi_patch], verbose=0)

#             # Assume patch_predictions[1] is for segmentation task
#             y_pred[y:y + patch_height, x:x + patch_width] += patch_predictions[1][0]
#             count_map[y:y + patch_height, x:x + patch_width] += 1

#             pbar.update(1)
    
#     pbar.close()
#     averaged_predictions = y_pred / count_map
#     final_pred = np.argmax(averaged_predictions, axis=-1)

#     # Clip the predicted raster using the vector boundary
#     def clip_raster_with_vector(raster_array, profile, vector_path, output_path):
#         with rio.Env():
#             with rio.open(output_path, 'w', **profile) as dst:
#                 dst.write(raster_array.astype(rio.int8), 1)
#             clipper = gpd.read_file(vector_path)
#             with rio.open(output_path) as src:
#                 clipped, clipped_transform = mask(src, clipper.geometry, crop=True)
#                 profile.update({
#                     'height': clipped.shape[1],
#                     'width': clipped.shape[2],
#                     'transform': clipped_transform
#                 })
#                 with rio.open(output_path, 'w', **profile) as dst:
#                     dst.write(clipped)

#     # Save the predicted array as a georeferenced raster and then clip
#     with rio.Env():
#         profile = src.profile
#         profile.update(
#             dtype=rio.int8,
#             count=1,
#             width=final_pred.shape[-1],
#             height=final_pred.shape[-2],
#             transform=src.transform,
#             compress='lzw'
#         )
#         temp_pred_path = 'temp_prediction.tif'
#         with rio.open(temp_pred_path, 'w', **profile) as dst:
#             dst.write(final_pred.astype(rio.int8), 1)

#         # Clip the temporary prediction raster using the city boundary
#         clip_raster_with_vector(final_pred, profile, vector_path, prediction_path)

#     print(f"Prediction map saved and clipped successfully to {prediction_path}")


In [ ]:
# import rasterio as rio
# import numpy as np
# from tqdm import tqdm
# from rasterio.mask import mask
# import geopandas as gpd

# mtcnn_inference(
#     N_CLASSES=3, 
#     raster_paths=(f'./dataset/{city}/original/2019/S2_2019_SR.tif'), 
#     ndbi_path=(f'./dataset/{city}/original/Density.tif'), 
#     model=model, 
#     prediction_path=f'./prediction/{city}/{city}_{inputs_str}_{model.name}.tif',
#     vector_path='/data/training_data/city_boundary/nairobi_admin_boundary.geojson'
# )